In [2]:
# we can convert the rules into before, after dictionaries
# then also convert each update into a list format
# iterate over each element of each update
# then check the two sides of the list from the element with the rule dictionaries

def parse_input(input_str: str):
    rules_str, updates = input_str.split('\n\n')
    # parse updates into list format
    update_list = []
    for update_str in updates.split('\n'):
        update = []
        for page in update_str.split(','):
            update.append(int(page))
        update_list.append(update)

    # print(update_list)

    # parse rules into dictionaries
    
    # before_dict: keys must come before values
    before_dict = dict()
    # after_dict: keys come after values
    after_dict = dict()

    for line in rules_str.split('\n'):
        [before, after] = [int(page) for page in line.split('|')]
        if before in before_dict:
            before_dict[before].append(after)
        else:
            before_dict[before] = [after]
        if after in after_dict:
            after_dict[after].append(before)
        else:
            after_dict[after] = [before]

    # print(before_dict)
    # print(after_dict)
    
    return update_list, before_dict, after_dict

class Manual:
    def __init__(self, update_list: list, before_rules: dict, after_rules: dict):
        self.update_list = update_list
        self.before_rules = before_rules
        self.after_rules = after_rules

    def checkUpdate(self, update: list):
        for index, element in enumerate(update):
            # splice the list into two parts, before and after element
            if index > 0:
                pre = update[:index] # list of numbers in the update set that come before the current num 
                # hence, none of these same numbers should be in the corresponding before_dict value
                for item in pre:
                    if item in self.before_rules.get(element, []):
                        return False
        
            if index < len(update)-1:
                post = update[index+1:]
                for item in post:
                    if item in self.after_rules.get(element, []):
                        return False

        return True
    
    def getMiddleNumber(self, update: list):
        # assumption is that list is of odd number length
        middle_index = int(len(update) / 2)
        return update[middle_index]
    
    def part1(self):
        valid_updates = [update for update in self.update_list if self.checkUpdate(update)]
        sum_of_middle_nums = sum([self.getMiddleNumber(update) for update in valid_updates])
        return sum_of_middle_nums

In [16]:
with open('data/test/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual(update_list, before_dict, after_dict)
manual.part1()


[[75, 47, 61, 53, 29], [97, 61, 53, 29, 13], [75, 29, 13], [75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]]
{47: [53, 13, 61, 29], 97: [13, 61, 47, 29, 53, 75], 75: [29, 53, 47, 61, 13], 61: [13, 53, 29], 29: [13], 53: [29, 13]}
{53: [47, 75, 61, 97], 13: [97, 61, 29, 47, 75, 53], 61: [97, 47, 75], 47: [97, 75], 29: [75, 97, 53, 61, 47], 75: [97]}


143

In [17]:
with open('data/input/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual(update_list, before_dict, after_dict)
manual.part1()

[[52, 77, 83, 31, 94, 75, 34, 95, 29, 38, 82, 19, 41, 39, 27, 98, 84, 13, 55, 21, 66], [76, 17, 43, 93, 46, 67, 68, 28, 18, 32, 87, 15, 89, 27, 31], [28, 63, 89, 27, 13, 66, 98, 95, 19, 36, 55], [38, 21, 39, 86, 16, 62, 76, 85, 47, 35, 93, 67, 68, 28, 18], [46, 67, 68, 28, 18, 87, 15, 89, 27, 31, 52, 13, 75, 66, 29, 98, 95, 19, 44, 77, 84], [41, 34, 98, 29, 95, 96, 55, 86, 52, 13, 16, 38, 82], [55, 83, 21, 16, 62, 76, 56, 43, 46], [37, 14, 56, 17, 18, 16, 35, 67, 21, 38, 59, 85, 93], [63, 18, 55, 27, 98, 32, 84, 19, 89, 87, 52, 44, 31, 95, 77, 36, 66, 13, 28, 29, 15, 94, 75], [55, 76, 39, 16, 82, 19, 34, 66, 29, 96, 75], [77, 34, 84, 94, 36, 55, 83, 82, 41, 38, 21, 39, 96, 86, 16, 62, 76, 56, 85, 59, 47, 35, 37], [62, 17, 47, 39, 37, 86, 85, 74, 38, 14, 93, 16, 67], [37, 67, 56, 46, 85, 62, 43, 68, 96, 17, 86, 39, 14, 76, 41, 82, 16], [89, 44, 67, 98, 28, 13, 75, 31, 63, 87, 52, 84, 34, 68, 77, 32, 46], [41, 38, 39, 96, 62, 17, 56, 47, 35, 37, 93], [44, 84, 94, 36, 21, 16, 62, 76, 85, 

5091

In [ ]:
#part2
from collections import deque
from tqdm.notebook import tqdm

class Manual2(Manual):
    def fixUpdate(self, update: list):
        sorted_list = []
        # pop numbers off a deque
        # if number is in order, add to new list 
        # if number not in order, add to other end of deque
        # loop over deque until solved 

        dq = deque()
        for item in update:
            dq.append(item)

        while dq:
            a = dq.pop()
            # check if a is in order (ie. everything before it in the deque is legally there)
            # if not in order, move to other end of deque
            # if in order, move to sorted list
            in_order = True
            for item in self.before_rules.get(a, []): # comparing what should come after the popped item with the set of numbers that actually come before the popped item in the deque
                if item in dq:
                    dq.appendleft(a)
                    in_order = False
                    break
            if in_order:
                sorted_list.append(a)

        return sorted_list

    def part2(self):
        invalid_updates = [update for update in self.update_list if not self.checkUpdate(update)]
        print(invalid_updates)
        fixed_updates = [self.fixUpdate(update) for update in invalid_updates]
        print(fixed_updates)
        sum_of_middle_nums = sum([self.getMiddleNumber(update) for update in fixed_updates])
        return sum_of_middle_nums



In [60]:
with open('data/test/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual2(update_list, before_dict, after_dict)
manual.part2()

[[75, 97, 47, 61, 53], [61, 13, 29], [97, 13, 75, 29, 47]]
[[53, 61, 47, 75, 97], [13, 29, 61], [13, 29, 47, 75, 97]]


123

In [61]:
with open('data/input/5.txt', 'r', encoding='utf-8') as f:
    input_str = f.read()

update_list, before_dict, after_dict = parse_input(input_str)
manual = Manual2(update_list, before_dict, after_dict)
manual.part2()

[[52, 77, 83, 31, 94, 75, 34, 95, 29, 38, 82, 19, 41, 39, 27, 98, 84, 13, 55, 21, 66], [41, 34, 98, 29, 95, 96, 55, 86, 52, 13, 16, 38, 82], [37, 14, 56, 17, 18, 16, 35, 67, 21, 38, 59, 85, 93], [63, 18, 55, 27, 98, 32, 84, 19, 89, 87, 52, 44, 31, 95, 77, 36, 66, 13, 28, 29, 15, 94, 75], [55, 76, 39, 16, 82, 19, 34, 66, 29, 96, 75], [62, 17, 47, 39, 37, 86, 85, 74, 38, 14, 93, 16, 67], [37, 67, 56, 46, 85, 62, 43, 68, 96, 17, 86, 39, 14, 76, 41, 82, 16], [89, 44, 67, 98, 28, 13, 75, 31, 63, 87, 52, 84, 34, 68, 77, 32, 46], [34, 13, 32, 83, 18, 89, 94, 31, 84, 27, 29, 95, 19, 75, 55, 63, 44, 52, 66, 98, 77], [75, 68, 67, 89, 74, 15, 28, 63, 44, 98, 29, 93, 52, 18, 95, 43, 13], [66, 75, 29, 39, 36, 82, 19, 13, 95, 44, 84, 96, 38, 16, 34, 52, 55, 86, 98, 83, 21], [38, 39, 37, 93, 17, 59, 68, 82, 85, 46, 47, 16, 67, 86, 62, 14, 35], [75, 34, 27, 98, 89, 28, 84, 63, 36, 32, 87, 55, 31], [55, 21, 17, 56, 83, 98, 76, 62, 39, 38, 19, 29, 44, 41, 84, 34, 95, 86, 16], [95, 27, 19, 77, 98, 31, 84

4681